# Steam Palette Extractor
Reference: https://github.com/woctezuma/steam-palette-extractor

## Install Python packages

In [ ]:
%cd /content
!git clone https://github.com/woctezuma/steam-palette-extractor.git
%cd /content/steam-palette-extractor
%pip install -qq -r requirements.txt

## Download images from Steam (only the first time)

In [ ]:
from src.constants import APPID_FNAME

GITHUB_URL = "https://github.com/woctezuma/steam-palette-extractor/releases"

!curl -OL {GITHUB_URL}/download/games/{APPID_FNAME}

In [ ]:
from src.constants import IMG_FOLDER
from src.download_utils import write_to_text_file
from src.utils import get_app_ids

TEMPORARY_FILE = "myimglist.txt"

write_to_text_file(get_app_ids(), fname=TEMPORARY_FILE)

# The download process took ~ 30 minutes.
# Out of 95,800 images, 92,249 were successfully downloaded.
# The output folder uses ~ 8 GB of disk space.
!echo img2dataset --url_list={TEMPORARY_FILE} --output_folder={IMG_FOLDER} --resize_mode=no

## Check the content of the image folder

In [ ]:
from src.filter_utils import prepare_filtered_files
from src.utils import get_app_ids, get_test_fnames

test_fnames = get_test_fnames(f'{IMG_FOLDER}/*')
print(f'#images = {len(test_fnames)}')

prepare_filtered_files(get_app_ids, test_fnames)

## Compute the palette for each Steam game

In [ ]:
from src.extract_utils import extract_from_scratch

pre_computed_palettes = extract_from_scratch(test_fnames)

## Load pre-computed data

In [ ]:
from src.constants import FILTERED_APP_IDS_FNAME, PALETTE_FNAME

!curl -OL {GITHUB_URL}/download/colors/{FILTERED_APP_IDS_FNAME}
!curl -OL {GITHUB_URL}/download/colors/{PALETTE_FNAME}

In [ ]:
from src.utils import get_filtered_app_ids, get_pre_computed_palettes

pre_computed_app_ids = get_filtered_app_ids()
pre_computed_palettes = get_pre_computed_palettes()

## Load data intended to evaluate the results

In [ ]:
from src.constants import APPID_FNAME, POPULAR_APPIDS_FNAME, SOLUTIONS_FNAME

GITHUB_URL_FOR_POPULARITY = "https://github.com/woctezuma/steam-popular-appids/releases"

!curl -OL {GITHUB_URL}/download/solutions/{SOLUTIONS_FNAME}
!curl -o {POPULAR_APPIDS_FNAME} -L {GITHUB_URL_FOR_POPULARITY}/download/data/{APPID_FNAME}

In [ ]:
from src.utils import get_egs_solutions, get_popular_appids

egs_solutions = get_egs_solutions()
popular_appids = get_popular_appids()

## Run the workflow

In [ ]:
from src.constants import get_default_params

params = get_default_params()
print(params)

### Define the target

In [ ]:
from src.image_utils import prepare_image
from src.url_utils import from_gift_to_egs_url

gift_index = 12
gift = egs_solutions["gift"][gift_index]

path_or_url = from_gift_to_egs_url(egs_solutions, gift)
reference_colors = prepare_image(path_or_url, params)

### Check the ground truth

In [ ]:
from src.distance_utils import compute_distance_between_palettes
from src.download_utils import get_image_url
from src.image_utils import prepare_image

# There can be several appIDs for different editions of a game, e.g. GOTY.
for ground_truth_app_id in gift["appids"]:
  path_or_url = get_image_url(ground_truth_app_id)
  ground_truth_colors = prepare_image(path_or_url, params)

  distance = compute_distance_between_palettes(
      reference_colors,
      ground_truth_colors,
      params,
      )

  print(f'\tappID: {ground_truth_app_id} ; distance: {distance:.2f}')

### Check all

#### Constrain the results to popular apps

In [ ]:
MAX_NUM_POPULAR_APP_IDS = 20000

test_app_ids = popular_appids[:MAX_NUM_POPULAR_APP_IDS]

# We constrain the number of appIDs (originally ~ 100k) to focus on games which
# may be able to attract the attention of Epic Games in order to strike a deal
# for a giveaway.
# This step is not mandatory, but it should help to make the whole process
# faster, and make the game of interest appear at lower ranks in the results.
# This means that it is easier to manually parse the results, typically by
# looking at the top 20 results instead of the top 100 results.

# - With the first 2,000 popular appIDs, 12 apps can be recalled out of 22 apps.
# - With the first 7,000 popular appIDs, 16 apps.
# - With the first 13,000 popular appIDs, 19 apps.
# - With the first 18,500 popular appIDs, 21 apps.
# NB: the missing app is the DLC for Destiny 2, which cannot be recovered anyway
# as it is not a game. However, the base game appears in the 21 recalled apps.

#### Run

In [ ]:
from src.distance_utils import compute_distances_with_all_the_palettes, get_ground_truth_ranks, get_most_similar_app_ids

gift_index = 12
verbose = True

gift = egs_solutions["gift"][gift_index]
path_or_url = from_gift_to_egs_url(egs_solutions, gift)

reference_colors = prepare_image(path_or_url, params, verbose=verbose)

distance_dict = compute_distances_with_all_the_palettes(
    reference_colors,
    pre_computed_palettes,
    pre_computed_app_ids,
    test_app_ids,
    params,
    verbose=verbose,
    )

most_similar_app_ids = get_most_similar_app_ids(distance_dict)

ground_truth_ranks = get_ground_truth_ranks(
    gift["appids"],
    most_similar_app_ids,
    )

#### Show the covers with the most similar color palettes

In [ ]:
from src.display_utils import display_results

max_num_displayed_images = 11

display_results(
    most_similar_app_ids,
    distance_dict,
    max_num_displayed_images,
    )

## Optimize the parameters

In [ ]:
MAX_NUM_POPULAR_APP_IDS = 2000

test_app_ids = popular_appids[:MAX_NUM_POPULAR_APP_IDS]

test_egs_solutions = {
    "image": egs_solutions["image"],
    "gift": [
        e
        for e in egs_solutions["gift"]
        if any(str(app_id) in test_app_ids for app_id in e["appids"])
        ],
}

In [ ]:
import torch

from src.optimize_utils import process_every_gift

def objective(trial):
  params = get_default_params()
  params["exponent"] = trial.suggest_float("exponent", 0.0, 1.0)
  params["factor"] = trial.suggest_float("factor", 0.0, 1.0)

  gift_ranks = process_every_gift(
      test_egs_solutions,
      pre_computed_palettes,
      pre_computed_app_ids,
      test_app_ids,
      params,
      verbose=False,
      )

  l = torch.Tensor([r for r in gift_ranks if r is not None])
  print(f"#gifts = {l.size()[0]} ---> min = {l.min():.0f} ; median = {l.median():.0f} ; mean = {l.mean():.2f} ; max = {l.max():.0f}")
  print(f"Parameters: {params}")
  print(f"Constraint: the top {MAX_NUM_POPULAR_APP_IDS} popular apps.")
  print('\n---\n')

  score = l.min() + l.median() + l.mean() + l.max()

  return score

In [ ]:
import joblib
import optuna

from pathlib import Path

STUDY_FNAME = "study.pkl"

NUM_TRIALS = 10
TIMEOUT_IN_SECONDS = 3600

if Path(STUDY_FNAME).exists():
  study = joblib.load(STUDY_FNAME)
  # https://optuna.readthedocs.io/en/stable/faq.html#how-can-i-save-and-resume-studies
  print("Best trial until now:")
  print(" Value: ", study.best_trial.value)
  print(" Params: ")
  for key, value in study.best_trial.params.items():
      print(f"    {key}: {value}")
else:
  study = optuna.create_study()

study.optimize(objective, n_trials=NUM_TRIALS, timeout=TIMEOUT_IN_SECONDS)
print(f"Best params is {study.best_params} with value {study.best_value}")

joblib.dump(study, STUDY_FNAME)